___
### <center> <u> <h1> <strong> Utilisation de la bibliothèque SciPy </strong> </h1> </center>

<h3>
Dans ce notebook on verra comment utiliser la bibliothèque Scipy pour :
    
<ol>
    <li> Gérer des problèmes d'optimisation </li>
    <li> Interpoler des fonctions </li>
    <li> Faire un calcul d'intégration numérique </li>
    <li> Résoudre des équations différentielles </li>
    <li> Réaliser des transformée Fourier </li>
    <li> Resoudre des équation linéaires </li>
    <li> Résoudre des problèmes destatistique </li>

</ol>
</h3>
    
---

<h1> <strong> A - Importation des modules necessaires

In [1]:
################################################################################
##################  Bibliothèque utilitaire mathémlatiques  ####################
################################################################################
import numpy as np
import scipy as sp
################################################################################
##################         Bibliothèque graphique           ####################
################################################################################
import matplotlib.pyplot as plt


<h1> <strong> B - Problème d'optimisation

<h1> <strong> B.1 - Simple problème de minimisation

<h3> <p> Supposons que nous soyons face à un problème dans lequel on a besoin e minimiser une grande donnée en Physique, Mahtémaque, Biologie . . </p>
    <p> On peut par exemple être comptable dans une entreprise donnée et on souhaite trouver le Mois sur les 10 dernière années où le chiffre d'affaire de l'entreprise a été au plus bas. Par ailleurs supposons que ce chifffre d'affaire soit modélisé par la fonction 

$f(t) = (t-3)^{2.6} + 5(t-3)^{5} $ </p>
    <p> Le problème se traduit donc à minimiser la fonction en question
   

In [2]:
def f( t ) :
    return (t-3)**2.6 + 5*(t-3)**5

<h3> Pour se faire, la bibliothèque Scipy offre la possibilité de minimiser des fonctions grâce àla méthode $minimize$ de son module $optimize$

In [ ]:
from scipy.optimize import minimize

<h3> La documentation de la fonction peut être parcourue

In [ ]:
minimize?

<h3> Comme on peut le voir, pour trouver le minimum en  paramètres de base  il suffit de fournir à la fonction, l'indicatif de la fonction à minimiser et une valeur de départ.

In [ ]:
result = minimize( f , x0 = 3.1)

In [ ]:
result

In [ ]:
type(result)

<h3> La fonction retourne alors un objet qui contient toutes les informations necessaires sur la réponse au problème de minimisation <ul>  
    <li> $result.x$ donne la liste de valeurs de la fonction qui minimise cette dernière; résultat sous forme de list </li>
    <li> $result.fun$ donne la valeur minimale de la question </li>
    <li> $result.status$, $result.success$ et $result.message$ donnent des informations sur le statut de la requête de minimisation </li>
    </ul>

In [ ]:
result.x[0]

In [ ]:
result.fun

In [ ]:
result.success

In [ ]:
result.status

In [ ]:
result.message

<h3> Comment on pouvait s'y attendre :<ul> 
    <li> La fonction est minimale pour une valeur proche de 3 </li>
    <li> La valeur minimale de la fonction est proche de 0  </li>
    </ul>

<h1> <strong> B.1 - Problème de minimisation avec contraintes

<h3> <p>La même fonction pourrait être utilisée pour traiter un problème de minimisation imposant plusieurs contraintes sur les variables du problème.</p>
    <p>  Imaginons à présent que le chiffre d'affaire de l'entreprise soit plutot perçu comme une fonction qui ne dépende pas uniquement du nombre de ses employés, de la qualité du service ou des produits qu'elle fourni à la société et de son budget marketing. </p>
    <p>On notera $n$ le nombre d'enployé, $q$ la qualité du service ou des produits qu'elle fourni ou vend et $b$ son budget marketing </p>
    <p> Supposons ensuite que son chiffre d'affaire soit modélisable par la fonction analytique suivante     
    <p> $f(n, q, b) = 10nq + 5(2b - 150q)^{4} + (b - 7qn^{2})^{6}$ </p>
    <p> La question se pose alors de savoir quelle est le chiffre d'affaire minimile que l'entreprise pourrait espérer avoir avec moins de 10 employés, un budget de  inférieur à 10000 € et avec une contrainte supplémentaire qui soit que

$50np - b \geq 0$

- ### <strong> La fonction chiffre d'affaire

In [ ]:
def f ( n , q , b) :
    return 10*n*q + 5*(2*b - 150*q)**4 + (b-7*q*n**2)**6

- ### <strong> Les contraintes du problème

- ### <strong> Les valeurs autorisées aux variables

---

- ## <strong> Authors </strong>

 #### <ul> <p> - <strong><a href="https://www.linkedin.com/in/brice-kengni-zanguim/" target="_blank">Brice KENGNI ZANGUIM :</a> </strong></p> <ul> Artificial Intelligence Engeneer at home and holds a PhD in Nano-physic. His research focused on using <strong>Machine Learning, Deep Learning, Signal Processing, Natural langage Processing, Computer Vision and so one</strong> to figure out how the world behave and impact human cognition. Brice KENGNI ZANGUIM has been working at home on mastering his own AI projects in many domains while seeking for a position in a company. </ul></ul>


---

- ## <strong> Other Contributors </strong>

#### <ul>None</ul>

---

- ## <strong> Change Log </strong>

|  Date (DD-MM-YYYY) |  Version | Changed By  |  Change Description |  Purpose  |
|---|---|---|---|---|
| 19- 03 - 2023 | 1.0 | Brice KENGNI ZANGUIM | Updated the file | Mem,save |
|-|-|-|-|
|   |   |   |   |   |

### <h3 align="center"> © BKZ Corporation 2023. All rights reserved. <h3/>
---